In [8]:
# Import libraries
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 500)
pd.options.display.max_colwidth = 1000
from scipy.sparse.linalg import spsolve
import scipy.sparse as sparse
import seaborn as sns

In [11]:
# Import csv file
train_path = "trivagoRecSysChallengeData2019_v2/train.csv"
test_path = "trivagoRecSysChallengeData2019_v2/test.csv"

In [15]:
# Create DataFrames
trivago_df = pd.read_csv(train_path)
trivago_test_df = pd.read_csv(test_path)

In [6]:
# Sanity check
trivago_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [16]:
# Convert UNIX time stamp (timestime column) to UTC 
date_conv = lambda x: datetime.datetime.utcfromtimestamp(x).strftime("%Y-%m-%d")
time_conv = lambda x: datetime.datetime.utcfromtimestamp(x).strftime("%H:%M:%S")
trivago_df["date"] = trivago_df["timestamp"].map(date_conv)
trivago_df["time"] = trivago_df["timestamp"].map(time_conv)

In [17]:
trivago_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,date,time
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:57:40
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:58:42
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:58:42
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:58:52
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:58:52


In [18]:
trivago_df.drop("timestamp", axis = 1, inplace = True)

In [19]:
trivago_df.head()

,user_id,session_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices,date,time
0,00RL8Z82B2Z1,aff3928535f48,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:57:40
1,00RL8Z82B2Z1,aff3928535f48,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:58:42
2,00RL8Z82B2Z1,aff3928535f48,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:58:42
3,00RL8Z82B2Z1,aff3928535f48,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:58:52
4,00RL8Z82B2Z1,aff3928535f48,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,2018-11-01,01:58:52


In [24]:
# Create dataframe for sparse matrix
matrix_df = pd.DataFrame()
matrix_df = trivago_df[["user_id", "reference", "action_type"]]

In [25]:
# Sanity check
matrix_df.head()

,user_id,reference,action_type
0,00RL8Z82B2Z1,Newtown,search for poi
1,00RL8Z82B2Z1,666856,interaction item image
2,00RL8Z82B2Z1,666856,interaction item image
3,00RL8Z82B2Z1,666856,interaction item image
4,00RL8Z82B2Z1,109038,interaction item image


In [27]:
# Drop rows where action_type is not clickout - clickout will be the proxy for 
# popularity, since the dataframe only has implicit feedback.
matrix_df = matrix_df[(matrix_df["action_type"] == "clickout item")]

In [28]:
# Sanity check
matrix_df.head(10)

,user_id,reference,action_type
13,00RL8Z82B2Z1,109038,clickout item
15,00RL8Z82B2Z1,1257342,clickout item
115,02SRUT1NQYH1,2795374,clickout item
121,03K8AXBL4BX2,1032816,clickout item
122,03K8AXBL4BX2,1032816,clickout item
176,03P4VFKK12UO,65685,clickout item
177,03P4VFKK12UO,1320460,clickout item
180,0473FZ8UNXRS,3143258,clickout item
181,066TUPQWUEV5,2552514,clickout item
184,06S61EKCW1JY,110591,clickout item


In [29]:
# Sanity check
matrix_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586586 entries, 13 to 15932991
Data columns (total 3 columns):
user_id        1586586 non-null object
reference      1586586 non-null object
action_type    1586586 non-null object
dtypes: object(3)
memory usage: 48.4+ MB


In [30]:
# Check number of unique values
matrix_df.nunique()

user_id        717774
reference      289506
action_type         1
dtype: int64

In [46]:
# Replace "clikout item" for an int 1 to sum up ocurrences per user
f = lambda x: x.replace("clickout item", "1")
g = lambda x: int(x)
matrix_df["action_type"] = matrix_df["action_type"].map(f)
matrix_df["action_type"] = matrix_df["action_type"].map(g)

In [51]:
# groupby user and item, sum the number of clicks
matrix_df = matrix_df.groupby(["user_id","reference"])[["action_type"]].sum()

In [52]:
# Sanity check
matrix_df.head(100)

action_type
user_id      reference             
0001VQMGUI65 477811               1
             950829               1
             2019467              1
             3133074              2
0003QTCX5MJX 2195060              1
0004IOZI7CKF 110985               1
             2627602              1
             7822344              1
0004WCFRV3FB 1439375              1
0006W0R5A5V8 6776722              1
0008B0X0HC39 2762974              1
0008BO33KUQ0 452021               2
             1949601              2
             2143854              1
             9488044              1
0008W02TLRNR 3480564              1
000EKQEFMW6S 3249726              1
             5665184              1
000I430EXZC0 103452               1
000III2Y1FXN 2940405              1
000JF7EK3WYX 5962870              1
000LC1EWG7ZP 3502458              1
             3859896              1
             4060836              1
             4264758              1
             4809064              1
             5437766              2
000NATJBKR5O 154153               1
000O8CPC6T0W 15196                1
000REY2YYH1D 56901                1
000ZIDXUIGI9 2674214              1
000ZN9219G41 149015               1
0010P7RW3UB7 1751223              1
0010PD3FH7SV 5963554              1
             7988660              1
0011FVIBJ0CE 6687858              1
0011WEV5DABC 1339750              1
             4374166              1
             5069538              1
             5797214              1
0012IHLNTQWC 4079384              1
0013L641G5P2 94010                2
00153N8QJ6WT 87653                1
00157FRJ3Y5W 131689               1
0015CRBC6OH9 48847                1
             82322                1
             688021               1
0015J9UBMKV5 1543025              1
0017FIR55K7R 2745276              4
00180FGBX4P2 103341               1
0018EAWF0OIK 998597               2
0019V5KKYL2W 1363672              1
001BJ5U1AZHW 5175124              2
001E8LJNLKWA 9614652              1
001EBIZM062K 214161               1
001F4FO9P0ED 48064                1
001G88F8NP9P 972027               1
001I03QURZH9 47982                1
001IC3WK4AOO 92313                2
001JFV0AYE6Z 67196                1
001OBEBO5LFE 12959                1
001OCXQ9PXET 13857                1
             2450306              1
001P10N99L60 6319850              1
001Q2N2IXI18 54716                1
             1471853              3
001TEVEVUEBE 8415608              1
001TKZND4DXR 1236918              2
             5971616              1
001UXWLZZDQG 1212028              2
             5083160              1
001WJ8QTM41X 6557710              2
001WP37I7I1T 4658158              2
001YQ253U4DR 2728380              1
001YTM35MP54 137078               1
0020A5F42QQX 86992                1
0022E2NEUGQX 9399034              1
0022VNDYMS6H 862971               1
0023FC807SR8 1384030              1
             4341920              1
             5534760              1
             6230236              1
0023Q64AC95I 1769325              1
0024WQMRS5XT 3970868              1
             9250178              1
0025AKR8Y9XQ 7180738              2
0025B8BU0NYP 2829394              1
             3374414              1
00273S57RU2X 9633022              1
00280CNY7H6F 5213152              1
0029BRXGBS69 129382               1
             164085               1
             486816               4
             928711               1
             935801               1
             984971               1
             984973               1
             1174271              1
             1238515              2
             2103382              1

In [56]:
# Change column name
matrix_df["clicks"] = matrix_df["action_type"]
matrix_df.drop("action_type", axis = 1, inplace = True)

In [58]:
# Reset index
matrix_df = matrix_df.reset_index()

In [59]:
# Sanity check
matrix_df.head()

,user_id,reference,clicks
0,0001VQMGUI65,477811,1
1,0001VQMGUI65,950829,1
2,0001VQMGUI65,2019467,1
3,0001VQMGUI65,3133074,2
4,0003QTCX5MJX,2195060,1


In [63]:
# Get unique customers
users = list(np.sort(matrix_df.user_id.unique()))

# Get unique products that were purchased
properties = list(matrix_df.reference.unique())

# Get all clicks
clicks = list(matrix_df.clicks)

# Get row indices
rows = matrix_df.user_id.astype("category", categories = users).cat.codes

# Get column indices
cols = matrix_df.reference.astype("category", categories = properties).cat.codes

# Create sparse matrix
sparse_matrix = sparse.csr_matrix((clicks, (rows, cols)), shape = (len(users), len(properties)))

In [64]:
# Sanity check
sparse_matrix

<717774x289506 sparse matrix of type '<class 'numpy.int64'>'
	with 1306901 stored elements in Compressed Sparse Row format>

In [65]:
# Check sparsity of the sparse_matrix

# Number of possible interactions in the matrix
matrix_size = sparse_matrix.shape[0] * sparse_matrix.shape[1]

# Number of items interacted with
num_clicks = len(sparse_matrix.nonzero()[0])

# Sparsity percentage (needs to be less than 99%)
sparsity = 100 * (1 - (num_clicks / matrix_size))

print(sparsity)

99.99937107711408


In [ ]:
# Can't use collaborative filteringbecause matrix sparsity is over 99%